In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from imblearn import over_sampling

In [ ]:
leads = pd.read_csv("/data/leads_data.csv",  dtype= {'ga_lead_id':str, 'client_id':str})
target = pd.read_csv("/data/target_data.csv", dtype= {'lead_id':str, 'converted_to_tenant':bool})

<ipython-input-42-d242c4630440>:1: DtypeWarning: Columns (31,32,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  leads = pd.read_csv("/content/drive/MyDrive/ml/leads_data.csv",  dtype= {'ga_lead_id':str, 'client_id':str})


# Investigating target data

In [ ]:
#Looking at the raw data
print(target.describe(),'\n')
leads_unduped = leads.drop_duplicates()
target_unduped = target.drop_duplicates()
print(target_unduped.describe(),'\n')
target_unique = target.groupby("lead_id").nunique()
print(target_unique[target_unique['converted_to_tenant'] >= 2].count(), '\n')
print(target['converted_to_tenant'].value_counts(normalize=True), '\n')
print(target_unduped['converted_to_tenant'].value_counts(normalize=True) , '\n')

       lead_id converted_to_tenant
count   266046              266046
unique  198858                   2
top     347438               False
freq       471              185723 

       lead_id converted_to_tenant
count   214067              214067
unique  198858                   2
top     169128               False
freq         2              149136 

converted_to_tenant    15209
dtype: int64 

False    0.698086
True     0.301914
Name: converted_to_tenant, dtype: float64 

False    0.696679
True     0.303321
Name: converted_to_tenant, dtype: float64 



- We can see that there were repeat leads that were both converted and lost on different occasions.
- There is a number of repeated data (ex. lead 347438 having 471 different rows).
- The class split is largely unchanged before and after removing duplicate data.

With these observations, I can clean this dataframe by dropping the duplicate data rows and drop the 15k leads with 2 possible values to make our dataframe simpler.

# Investigating lead data

In [ ]:
#print(leads.columns) #two columns that are ids
target_lead_id = target['lead_id']
leads_client_id = leads[leads['client_id'].isin(target_lead_id)].drop_duplicates() #intersection between client_id and lead_id
leads_ga_id = leads[leads['ga_lead_id'].isin(target_lead_id)].drop_duplicates() #intersection between ga_lead_id and lead_id
print(leads_client_id.shape[0])
print(leads_ga_id.shape[0]) #this is our id

0
3643


In [ ]:
def custom_Describe (df):
  nonempty = []
  for column in df:
    print("------------------------------")
    print(column)
    print("Data type: ", df[column].dtype)
    print("N/A ratio: ", df[column].isna().mean())
    if df[column].nunique() <= 4:
      print("Value counts: ", df[column].value_counts())
      print("Unique values: ", df[column].unique())
    else:
      print("# of unique values: ", df[column].nunique())
    print("------------------------------")
    if df[column].isna().mean() < 0.3 and df[column].dropna().nunique() >= 2 :
      nonempty.append(column)
  print("Potential features: ", nonempty)


In [ ]:
leads['client_id'].value_counts()

2.07482E+19    6165
1.15932E+19    5030
4.6381E+18     1725
7.23042E+17    1699
1.82606E+16    1626
               ... 
5.56579E+17       4
1.92209E+19       4
1.09628E+19       4
5.08322E+18       3
2.09556E+19       3
Name: client_id, Length: 2938, dtype: int64

In [ ]:
leads['ga_lead_id'].value_counts()

288341    6
286258    6
283747    5
298617    4
293734    4
         ..
284656    1
284657    1
286607    1
292167    1
298986    1
Name: ga_lead_id, Length: 3842, dtype: int64

In [ ]:
#description before removing NaN lead_id
custom_Describe(leads)

------------------------------
client_id
Data type:  object
N/A ratio:  0.0
# of unique values:  2938
------------------------------
------------------------------
sessions
Data type:  int64
N/A ratio:  0.0
# of unique values:  5
------------------------------
------------------------------
timestamp
Data type:  object
N/A ratio:  0.7170089285714286
# of unique values:  72
------------------------------
------------------------------
hits
Data type:  float64
N/A ratio:  0.7170089285714286
# of unique values:  55
------------------------------
------------------------------
unique_events
Data type:  int64
N/A ratio:  0.0
# of unique values:  24
------------------------------
------------------------------
session_duration
Data type:  float64
N/A ratio:  0.7170089285714286
# of unique values:  2208
------------------------------
------------------------------
page_per_session
Data type:  float64
N/A ratio:  0.7170089285714286
Value counts:  0.0    74432
1.0     7975
Name: page_per_sessio

In [ ]:
#description after removing NaN lead_id
custom_Describe(leads.dropna(subset=['ga_lead_id']))

------------------------------
client_id
Data type:  object
N/A ratio:  0.0
# of unique values:  2461
------------------------------
------------------------------
sessions
Data type:  int64
N/A ratio:  0.0
Value counts:  0    4188
Name: sessions, dtype: int64
Unique values:  [0]
------------------------------
------------------------------
timestamp
Data type:  object
N/A ratio:  0.9749283667621776
# of unique values:  33
------------------------------
------------------------------
hits
Data type:  float64
N/A ratio:  0.9749283667621776
# of unique values:  14
------------------------------
------------------------------
unique_events
Data type:  int64
N/A ratio:  0.0
Value counts:  1    4051
0     137
Name: unique_events, dtype: int64
Unique values:  [1 0]
------------------------------
------------------------------
session_duration
Data type:  float64
N/A ratio:  0.9749283667621776
Value counts:  0.0    105
Name: session_duration, dtype: int64
Unique values:  [nan  0.]
-----------

There are a couple of columns that have too much missing data or too little unique data to be able to use. The ones that can still be useful and have potential to be useful derivatives:
- client_id
- unique_events
- event_action


Hypotheses:
1. Clients are assigned the same lead_id each time they become a lead.
2. Clients have a unique lead_id (ids are not reused).
3. Clients can have multiple interactions with the website.



In [ ]:
#Hypothesis 1
grouped_clients = leads.groupby(['client_id'])
same_id = (grouped_clients['ga_lead_id'].nunique() > 1)
same_id.mean() * 100 # 24.6% of clients have more than 1 unique id. They are not given the same id each time they become leads.

#False

24.642614023144997

In [ ]:
#Hypothesis 2
grouped_leads = leads_unduped.groupby(['ga_lead_id'])
more_than_one = grouped_leads['client_id'].nunique()
print((more_than_one>1).value_counts()) #3 leads with more than one client. We can just drop these since it's negligible.


#False

False    3839
True        3
Name: client_id, dtype: int64


In [ ]:
#Hypothesis 3

leads_unduped['client_id'].value_counts()
#True. Client can have as much as 1943 unique interactions.

2.07482E+19    1943
1.15932E+19    1356
1.88939E+18     561
1.26745E+19     516
1.20775E+19     494
               ... 
1.09628E+19       4
5.56579E+17       4
1.92209E+19       4
2.09556E+19       3
5.08322E+18       3
Name: client_id, Length: 2938, dtype: int64

# Data Cleaning



**Results:**
   - We can use "ga_lead_id" from leads_data to merge with target_data's "lead_id"
   - Clients are **NOT** assigned the same lead_id each time they become a lead.
   - Clients do **NOT** have a unique lead_id (lead_ids are reused). Only 3 of them are like this.
   - Clients can have multiple interactions with the website.
---
**Cleaning Methods:**
1. Drop the lead_ids from hypothesis 2.
2. Drop the unuseful columns described in custom_describe().
3. Drop Duplicates.
4. Drop lead_ids with more than one labels in target_data.
5. Sum the counts of unique events for each customer.
6. OHE for event_actions.




In [ ]:
#Removed duplicates and leads with more than one result
lead_filtered = leads_unduped.filter(items= ['client_id', 'unique_events', 'event_action', 'ga_lead_id'])
lead_filtered = lead_filtered.drop_duplicates()
lead_filtered['visit_freq'] = lead_filtered.groupby('client_id')['client_id'].transform('count')
lead_filtered = lead_filtered.dropna(subset=['ga_lead_id'])
lead_filtered = lead_filtered.rename(columns={"ga_lead_id":"lead_id"})

# Sum the counts of unique events for each customer.
lead_filtered['unique_events'] = lead_filtered.groupby('lead_id')['unique_events'].transform('sum')

#OHE for event_actions.
one_hot_encoded = pd.get_dummies(lead_filtered['event_action'])

# Group by lead_id and compress the one-hot encoding using max()
compressed_encoding = one_hot_encoded.groupby(lead_filtered['lead_id']).transform('max')
lead_filtered = pd.concat([lead_filtered, compressed_encoding], axis=1)
lead_filtered.drop(columns=['event_action'], inplace=True)
# Print the resulting dataframe
lead_filtered

,client_id,unique_events,lead_id,visit_freq,Inventory Commercial Lead,Inventory Residential Lead,Property Commercial Lead,Property Freehold Lead,Property Residential Lead,gtm.load,propertyFormSubmit
1497,1.10039E+19,1,283257,17,0,1,0,0,0,0,0
1593,1.6297E+19,1,291538,20,0,1,0,0,0,0,0
1788,1.10039E+19,1,283257,17,0,1,0,0,0,0,0
1813,1.94485E+19,1,290957,16,0,1,0,0,0,0,0
4508,3.15404E+18,1,283955,15,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
290496,3.22455E+18,2,299054,17,0,1,0,0,0,0,1
290497,3.22455E+18,2,299054,17,0,1,0,0,0,0,1
290665,5.16213E+18,2,298916,26,0,1,0,0,0,0,1
290666,5.16213E+18,2,298916,26,0,1,0,0,0,0,1


In [ ]:
#Removed duplicates and leads with more than one result
multilabel_ids =  target_unique[target_unique['converted_to_tenant'] > 1].index.tolist()
more_than_one_list = more_than_one[more_than_one>1].index.tolist()
target_filtered = target_unduped[~target_unduped['lead_id'].isin(multilabel_ids)]
target_filtered = target_filtered[~target_filtered['lead_id'].isin(more_than_one_list)]
target_filtered.shape


(183646, 2)

In [ ]:
#Final Clean Data

target_filtered
lead_filtered


,client_id,unique_events,lead_id,visit_freq,Inventory Commercial Lead,Inventory Residential Lead,Property Commercial Lead,Property Freehold Lead,Property Residential Lead,gtm.load,propertyFormSubmit
1497,1.10039E+19,1,283257,17,0,1,0,0,0,0,0
1593,1.6297E+19,1,291538,20,0,1,0,0,0,0,0
1788,1.10039E+19,1,283257,17,0,1,0,0,0,0,0
1813,1.94485E+19,1,290957,16,0,1,0,0,0,0,0
4508,3.15404E+18,1,283955,15,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
290496,3.22455E+18,2,299054,17,0,1,0,0,0,0,1
290497,3.22455E+18,2,299054,17,0,1,0,0,0,0,1
290665,5.16213E+18,2,298916,26,0,1,0,0,0,0,1
290666,5.16213E+18,2,298916,26,0,1,0,0,0,0,1


# Feature Engineering

Primitive features to generate:

*   count
* visit mean
* event ratio



In [ ]:
# generating primitive features
def count_unique_events(df):
    unique_events = df.groupby('client_id')['unique_events'].nunique().reset_index()
    unique_events.rename(columns={'unique_events': 'count_unique_events'}, inplace=True)
    return unique_events

def calculate_visit_mean(df):
    visit_mean = df.groupby('client_id')['visit_freq'].mean().reset_index()
    visit_mean.rename(columns={'visit_freq': 'visit_mean'}, inplace=True)
    return visit_mean

def calculate_event_ratio(df, event_type):
    event_ratio = df.groupby('client_id')[event_type].sum() / df.groupby('client_id')['unique_events'].nunique()
    event_ratio = event_ratio.reset_index().rename(columns={0: f'ratio_{event_type}'})
    return event_ratio

def generate_primitive_features(df):
    unique_events = count_unique_events(df)
    visit_mean = calculate_visit_mean(df)
    gtm_load_ratio = calculate_event_ratio(df, 'gtm.load')
    property_form_submit_ratio = calculate_event_ratio(df, 'propertyFormSubmit')

    primitive_features = unique_events.merge(visit_mean, on='client_id')
    primitive_features = primitive_features.merge(gtm_load_ratio, on='client_id')
    primitive_features = primitive_features.merge(property_form_submit_ratio, on='client_id')

    primitive_features = primitive_features.fillna(0)  # Replace missing values with 0


    return primitive_features

In [ ]:
def drop_constant_features(df):
    return df.loc[:, df.apply(pd.Series.nunique) != 1]

def drop_highly_correlated_features(df, threshold=0.9):
    correlation_matrix = df.corr().abs()
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(columns=to_drop)

def drop_duplicate_data_points(df):
    duplicate_indices = df.duplicated(subset=df.columns.difference(['lead_id', 'client_id', 'converted_to_tenant']))
    return df[~duplicate_indices]


In [ ]:
primitive_features_df = generate_primitive_features(lead_filtered)
primitive_features_df
lead_features = pd.merge(lead_filtered, primitive_features_df, on='client_id', how='inner')
lead_target_features = pd.merge(lead_features, target_filtered, on='lead_id', how='inner')
# Feature cleaning
lead_target_features = drop_constant_features(lead_target_features)
lead_target_features = drop_highly_correlated_features(lead_target_features, threshold=0.9)
# Final data with features
lead_target_features.drop_duplicates(inplace=True)
lead_target_features

<ipython-input-57-9ffd43cb8c0a>:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr().abs()
<ipython-input-57-9ffd43cb8c0a>:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))


,client_id,unique_events,lead_id,visit_freq,Inventory Residential Lead,Property Freehold Lead,gtm.load,propertyFormSubmit,count_unique_events,converted_to_tenant
0,1.10039E+19,1,283257,17,1,0,0,0,1,False
2,1.6297E+19,1,291954,20,1,0,0,0,1,True
3,1.6297E+19,1,291537,20,1,0,0,0,1,True
4,1.94485E+19,1,290957,16,1,0,0,0,1,False
5,1.94485E+19,1,290956,16,1,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...
3451,2.33817E+17,1,298790,27,1,0,0,0,1,False
3452,2.58971E+18,1,298769,16,1,0,0,0,1,False
3453,3.22455E+18,2,299054,17,1,0,0,1,1,True
3455,5.16213E+18,2,298916,26,1,0,0,1,1,False


# Model Selection

Models that we will start with are:
- **Logistic Regression**: Good baseline model
- **Random Forest**: Performs best well without much hyperparameter tuning
- **Gradient Boosting** (XGBoost): Known for good accuracy and performance on structured data.
- **SVM**: Suitabke for binary classification tasks and can handle linear and non-linear relationships
- **Naive Bayes**: Simple and fast for text-based classification.

These are known reliable models used for classification tasks.

In [ ]:
#Readying data for model use.
X = lead_target_features.drop(columns=['converted_to_tenant'])
y = lead_target_features['converted_to_tenant']


# Split the data into training and testing sets (80% train + validation, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training + validation set into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Verify the shapes of the split datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2133, 9)
X_test shape: (667, 9)
y_train shape: (2133,)
y_test shape: (667,)


In [ ]:
X_train = X_train.drop(columns=['client_id', 'lead_id'])

X_valid_index = X_val[['client_id', 'lead_id']]
X_test_index = X_test[['client_id', 'lead_id']]

X_val = X_val.drop(columns=['client_id', 'lead_id'])
X_test = X_test.drop(columns=['client_id', 'lead_id'])
# Oversampling using SMOTE
oversampling = over_sampling.SMOTE(random_state=42)
X_train, y_train = oversampling.fit_resample(X_train, y_train)

print("shape of train data:", X_train.shape, y_train.shape)

shape of train data: (3536, 7) (3536,)


In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'SVM': SVC(probability=True),
    'Naive Bayes': GaussianNB()
}

def evaluate_model(model, X_val, y_val):
    # Use cross-validation to obtain evaluation metrics on the validation set
    y_pred_val = cross_val_predict(model, X_val, y_val, cv=5)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_val, y_pred_val)
    precision = precision_score(y_val, y_pred_val)
    recall = recall_score(y_val, y_pred_val)
    f1 = f1_score(y_val, y_pred_val)
    auc_roc = roc_auc_score(y_val, y_pred_val)

    # Return the evaluation metrics as a dictionary
    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'AUC-ROC': auc_roc
    }



In [ ]:
model_scores = {}

# Evaluate each model using cross-validation on the validation set
for model_name, model in models.items():
    print(f"Evaluating {model_name}")
    model_scores[model_name] = evaluate_model(model, X_val, y_val)


scores_df = pd.DataFrame(model_scores)
print(scores_df)

Evaluating Logistic Regression
Evaluating Random Forest
Evaluating XGBoost
Evaluating SVM
Evaluating Naive Bayes
           Logistic Regression  Random Forest   XGBoost       SVM  Naive Bayes
Accuracy              0.784644       0.782772  0.771536  0.792135     0.492509
Precision             0.444444       0.476190  0.409091  1.000000     0.237458
Recall                0.035088       0.175439  0.157895  0.026316     0.622807
F1-score              0.065041       0.256410  0.227848  0.051282     0.343826
AUC-ROC               0.511591       0.561529  0.547995  0.513158     0.539975



Looking at the results we can see that the overall best performing model is the Random forest model.

# Model Training & Testing

In [ ]:
# Finding the best params
X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = pd.concat([y_train, y_val], axis=0)


param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf_model = RandomForestClassifier()

grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_val, y_train_val)


best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

# Train the best model on the combined training and validation data
best_rf_model.fit(X_train_val, y_train_val)

# Evaluate the best model on the test set
y_pred_test = best_rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)
test_auc_roc = roc_auc_score(y_test, y_pred_test)

# Print the best hyperparameters and test results
print("Best Hyperparameters:", best_params)
print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1-score:", test_f1)
print("Test AUC-ROC:", test_auc_roc)

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 150}
Test Accuracy: 0.7061469265367316
Test Precision: 0.30994152046783624
Test Recall: 0.40458015267175573
Test F1-score: 0.35099337748344367
Test AUC-ROC: 0.5922154494702062


# Benchmarking
Scoring the data.

In [ ]:
# Predict probabilities of being converted to a tenant for each lead in the test set
y_probs_test = best_rf_model.predict_proba(X_test)[:, 1]

# Create a DataFrame to store the leads with their corresponding probabilities
leads_with_probs = pd.DataFrame({'Lead_ID': X_test_index['lead_id'], 'Client_ID': X_test_index['client_id'], 'Probability': y_probs_test, 'Ground Truth': y_test})

# Sort the leads based on the probabilities in descending order to get the top picks
scoring = leads_with_probs.sort_values(by='Probability', ascending=False)

print("Most likely to convert to lead:")
print(scoring.head(10))

Most likely to convert to lead:
     Lead_ID    Client_ID  Probability  Ground Truth
3144  298534  7.45021E+18     1.000000          True
251   296658  1.72796E+19     1.000000          True
2548  297195  3.29294E+18     1.000000          True
3133  298536  7.45021E+18     1.000000          True
2044  291767  1.57364E+18     1.000000          True
3148  298535  7.45021E+18     1.000000          True
732   290874   1.5517E+19     0.953965          True
2195  296277   2.5867E+18     0.923054          True
2193  296274   2.5867E+18     0.923054          True
2199  296272   2.5867E+18     0.923054          True


In [ ]:
#Success rate without using a machine learning model
success_no_model = y_test.mean()*100
success_percentage = 100 * scoring.iloc[:100]["Probability"].mean()
print(f"Sucess of turning leads into tenants without using a model: {success_no_model}%")
print(f"Sucess of turning leads into tenants using a Random Forest model on the top 100 of the sorted list: {success_percentage}%")
print(f"Improvement rate: {success_percentage-success_no_model}%")


Sucess of turning leads into tenants without using a model: 19.640179910044978%
Sucess of turning leads into tenants using a Random Forest model on the top 100 of the sorted list: 65.43074751765035%
Improvement rate: 45.79056760760537%


We can see that the model we used provided a 46% improvement over manually choosing leads for the first 100 in our sorted list.